Script para replicar o algoritmo de Nelder Mead descrito por Lagarias et al 1999

In [1]:
using Distributed

In [2]:
nprocs = 3
addprocs(nprocs);

In [3]:
@everywhere using LinearAlgebra

In [4]:
# The six-hump camelback function:
# %  camel= @(x)(4-2.1*x(1).^2+x(1).^4/3).*x(1).^2+x(1).*x(2)+4*(x(2).^2-1).*x(2).^2;
# %  has a doble minimun at f(-0.0898,0.7126) = f(0.0898,-0.7126) = -1.0316
# %  this code works with it as follows:
# %  [x0,f0]=sim_anl(camel,[0,0],[-10,-10],[10,10],400)
# %  and we get:
# %  x0=[-0.0897 0.7126]

In [5]:
@everywhere function camel(x)
    
    return (4-2.1*x[1].^2+x[1].^4/3).*x[1].^2+x[1].*x[2]+4*(x[2].^2-1).*x[2].^2;
end

Parâmetros

In [6]:
r = 1 #reflection, denotado ρ
e = 2 #expansion, denotado χ
c = 1/2 #contraction, denotado γ
s = 1/2 #shrinkage, denotado σ

0.5

In [72]:
x0 = [- 0.08, 0.7]

n = size(x0,1)

f = camel

camel (generic function with 1 method)

In [73]:
pids = workers()

3-element Array{Int64,1}:
 2
 3
 4

Gerando simplexo inicial

In [107]:
#cada linha tem uma possível solução e existem n+1 colunas com possíveis soluções
x = zeros(n+1, n)

#o simplexo inicial vai ser o chute inicial expandido em cada direção
x[1,:] .= x0

for i in 2:n+1
    exp = ones(size(x0,1))
    exp[i-1] = e
    x[i,:] = x0 .* exp
end


Iterações

In [108]:
function reflection(xn, xw, n, r, f)
    #xn é o vetor com as n melhores entradas
    #xw é o ponto x_n+1, também conhecido aqui como x worst
    
    xm = transpose(sum(xn, dims = 1) )/n
    
    xr = xm .+ r * (xm .- xw)
    
    fr = f(xr)
    
    return xr, fr
    
end

reflection (generic function with 1 method)

In [109]:
function expand(xn, xr, n, e, f)
    
    xm = transpose( sum(xn, dims = 1)) /n
    
    xe = xm .+ e * (xr .- xm)
    
    fe = f(xe)
    
    return xe, fe
    
end
    

expand (generic function with 1 method)

In [110]:
function outside_contraction(xn, xr, n, c, f)
    
    xm = transpose( sum(xn, dims = 1)) /n
    
    xc = xm .+ c * (xr .- xm)
    
    fc = f(xc)
    
    return xc, fc
    
end

outside_contraction (generic function with 1 method)

In [111]:
function inside_contraction(xn, xw, n, c, f)
    
    xm = transpose( sum(xn, dims = 1)) /n
    
    xcc = xm .- c * (xm - xw)
    
    fcc = f(xcc)
    
    return xcc, fcc
    
end

inside_contraction (generic function with 1 method)

In [112]:
function shrink(x, n, s)
    
    #x precisa estar ordenado para funcionar aqui
    x1 = x[1,:]
    
    xs = zeros(size(x))
    xs[1, :] = x1
    
    for i in 2:n+1
        xs[i,:] = x1 + s * (x[i,:] - x1)
    end
    
    return xs
    
    
end

shrink (generic function with 1 method)

In [113]:
function evaluate(x, f, n, pids)
    #calcula a função nos pontos e retorna os pontos de interesse
    #f é a função que será avaliada
    #n é a dimensão do vetor de parâmetros
    #pids são os trabalhadores
    
    F = zeros(n+1)
    
    @sync for (i, w) in enumerate(pids)
        @async F[i] = @fetchfrom w f(x[i,:])
        println(w)
    end
    
#     for i in 1:n+1
#         F[i] = f(x[i,:])
#     end
    
    order = sortperm(F)
    F = F[order]
    x = x[order, :]
    
    xn = x[1:n, :]
    xw = x[n+1, :]
    
    f1 = F[1]
    fn = F[n]
    fw = F[n+1]
    
    
    #retorna os pontos de interesse para o algoritmo
    return x, xn, xw, f1, fn, fw
    
end
    

evaluate (generic function with 1 method)

In [114]:
#criar uma função para ajustar o valor inteiro no vetor x

In [115]:
#loop
#x, f, n, pids, convergência, número max de observações
#argumentos da função f

In [116]:
#estrutura do loop
#troquei n+1 por w, de "worst"

function interation(x, f, n, pids)
    #uma iteração do loop. baseado nas páginas 4 a 7 de Lagarias et al (1999)
    
    x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)

    xr, fr = reflection(xn, xw, n, r, f)

    if(f1 <= fr < fn)

        #accept reflected
        x[n+1, :] = xr

        println("reflected, xr = $xr")
        x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)

    elseif(fr < f1)

        #(fr < f1)
        #expand


        xe, fe = expand(xn, xr, n, e, f)

        if(fe < fr)

            #aceita xe e termina a iteração
            x[n+1, :] = xe

            println("expanded, xe = $xe")
            x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)



            else

            #(fe >= fr)
            #aceita xr e termina a iteração
            x[n+1, :] = xr

            println("reflected, xr = $xr")
            x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)

        end


    elseif(fr >= fn)

        #outside or inside contraction

        if(fn <= fr < fw)


            #outside contraction
            xc, fc = outside_contraction(xn, xr, n, c, f)

                if(fc <= fr)
                #aceita xc
                x[n+1, :] = xc

                println("outside contraction, xc = $xc")
                x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)

                else

                #(fc > fr)
                #shrink
                x = shrink(x, n, s)

                println("shrinked, xs = $x")
                x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)

                end



        elseif(fr >= fw)
            #inside contraction
            xcc, fcc = inside_contraction(xn, xw, n, c, f)

                if(fcc < fw)
                

                #aceita xcc
                x[n+1, :] = xcc

                println("inside contraction, xcc = $xcc")
                x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)

                else
                #(fcc > fw)
                #shrink
                x = shrink(x, n, s)

                println("shrinked, xs = $x")
                x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)
            end


        end

    end
    
    
    return x, xn, xw, f1, fn, fw
    
    
end

interation (generic function with 1 method)

In [117]:
function convergence(x, n, tol)
    #x é o vetor com parâmetros
    #n é a dimensão
    #tol é a tolerância, usaremos tol = 10^-6
    
    control = zeros(n)
    
    #olharemos a distância entre cada vetor e o melhor
    for i in 2:n+1
        control[i-1] = norm(x[1,:] - x[i,:]) < tol
    end
    
    if(sum(control) == n)#se todos os vetores estiverem próximos ao melhor, converge
        return 1
    else
        return 0
    end
    
    
end    

convergence (generic function with 1 method)

In [118]:
function nm(x, f, n, pids, tol, maxiter)
    
    k = 1
    converged = 0
    while(converged == 0 && k < maxiter)
        println("iteration $k")
        x, xn, xw, f1, fn, fw = interation(x, f, n, pids)
        converged = convergence(x, n, tol)
        
        k+=1
        
    end
    
    return x
    
end

nm (generic function with 1 method)

In [119]:
x_optimal = nm(x, f, n, pids, 10^-5, 1000)

iteration 1
2
3
4
outside contraction, xc = [-0.13999999999999999; 0.35]
2
3
4
iteration 2
2
3
4
inside contraction, xcc = [-0.13; 0.5249999999999999]
2
3
4
iteration 3
2
3
4
inside contraction, xcc = [-0.125; 0.6124999999999999]
2
3
4
iteration 4
2
3
4
outside contraction, xc = [-0.1175; 0.7437499999999999]
2
3
4
iteration 5
2
3
4
outside contraction, xc = [-0.068125; 0.7328124999999999]
2
3
4
iteration 6
2
3
4
inside contraction, xcc = [-0.09578125; 0.7300781249999999]
2
3
4
iteration 7
2
3
4
outside contraction, xc = [-0.0977734375; 0.7061523437499999]
2
3
4
iteration 8
2
3
4
inside contraction, xcc = [-0.09233398437500001; 0.7165771484374999]
2
3
4
iteration 9
2
3
4
inside contraction, xcc = [-0.08752685546875; 0.7056823730468749]
2
3
4
iteration 10
2
3
4
reflected, xr = [-0.08208740234375; 0.7161071777343748]
2
3
4
iteration 11
2
3
4
inside contraction, xcc = [-0.0873687744140625; 0.7110122680664062]
2
3
4
iteration 12
2
3
4
outside contraction, xc = [-0.09373336791992187; 0.71263

3×2 Array{Float64,2}:
 -0.0898397  0.712654
 -0.0898371  0.712658
 -0.0898483  0.712658

In [122]:
f(x_optimal[3,:])

-1.031628453315627

In [94]:
convergence(x_optimal, n, 10^-5)

0

In [99]:
x = x_optimal
tol = 10^-5

control = zeros(n)
    
#olharemos a distância entre cada vetor e o melhor
for i in 2:n+1
    control[i-1] = norm(x[1,:] - x[i,:]) < tol
end

In [100]:
control

2-element Array{Float64,1}:
 1.0
 1.0

In [ ]:
if(sum(control) == n)#se todos os vetores estiverem próximos ao melhor, converge
    return 1
else
    return 0
end
    

In [55]:
#estrutura do loop
#troquei n+1 por w, de "worst"

xr, fr = reflection(xn, xw, n, r, f)

if(f1 <= fr < fn)
    
    #accept reflected
    x[n+1, :] = xr
    
    println("reflected, xr = $xr")
    @show x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)
    
elseif(fr < f1)
    
    #(fr < f1)
    #expand
    
    
    xe, fe = expand(xn, xr, n, e, f)
        
    if(fe < fr)
        
        #aceita xe e termina a iteração
        x[n+1, :] = xe
        
        println("expanded, xe = $xe")
        @show x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)
        
        
        
        else
        
        #(fe >= fr)
        #aceita xr e termina a iteração
        x[n+1, :] = xr
    
        println("reflected, xr = $xr")
        @show x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)
        
    end
    
    
elseif(fr >= fn)
    
    #outside or inside contraction
    
    if(fn <= fr < fw)
        

        #outside contraction
        xc, fc = outside_contraction(xn, xr, n, c, f)
        
            if(fc <= fr)
            #aceita xc
            x[n+1, :] = xc

            println("outside contraction, xc = $xc")
            @show x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)
                
            else
            
            #(fc > fr)
            #shrink
            shrink(x, n, s)

            println("shrinked, xs = $xs")
            @show x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)
                
            end
            
        
            
    elseif(fr >= fw)
        #inside contraction
        xcc, fcc = inside_contraction(xn, xw, n, c, f)

            if(fcc < fw)
            
            #aceita xcc
            x[n+1, :] = xcc

            println("inside contraction, xcc = $xcc")
           @show x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)

            else
            #(fcc > fw)
            #shrink
            shrink(x, n, s)

            println("shrinked, xs = $xs")
            @show x, xn, xw, f1, fn, fw = evaluate(x, f, n, pids)
        end
        
        
    end
    
end

inside contraction, xcc = [-0.08983928528020402; 0.7126567110769588]
2
3
4
(x, xn, xw, f1, fn, fw) = evaluate(x, f, n, pids) = ([-0.08984334521676443 0.7126570891934809; -0.08983928528020402 0.7126567110769588; -0.08983966378507827 0.7126535348211465], [-0.08984334521676443 0.7126570891934809; -0.08983928528020402 0.7126567110769588], [-0.08983966378507827, 0.7126535348211465], -1.0316284534800175, -1.0316284534592501, -1.03162845340773)


([-0.08984334521676443 0.7126570891934809; -0.08983928528020402 0.7126567110769588; -0.08983966378507827 0.7126535348211465], [-0.08984334521676443 0.7126570891934809; -0.08983928528020402 0.7126567110769588], [-0.08983966378507827, 0.7126535348211465], -1.0316284534800175, -1.0316284534592501, -1.03162845340773)

In [60]:
#convergence control
norm(x[1,:] - x[2,:]) < 1^-6


true

In [62]:
a = zeros(1)

1-element Array{Float64,1}:
 0.0

In [63]:
a[1] = true

true

In [64]:
a

1-element Array{Float64,1}:
 1.0

In [67]:
sum(ones(2)) == 2

true

In [61]:
norm(x[1,:] - x[3,:]) < 1^-6

true

In [56]:
x

3×2 Array{Float64,2}:
 -0.0898433  0.712657
 -0.0898393  0.712657
 -0.0898397  0.712654

Próximos passos

- critério de convergência
- arredondar os valores para valores absolutos no caso dos parâmetros 5 e 6
- paralelizar no caso de shrink?